In [3]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.vectorstores import Pinecone
import os

In [4]:
loader=PyPDFDirectoryLoader("PDFs")
data=loader.load()

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
text_chunks = text_splitter.split_documents(data)

In [7]:
len(text_chunks)

91

In [29]:
from dotenv import load_dotenv
load_dotenv()
key = os.getenv("OPENAI_API_KEY")

In [30]:
embedding=OpenAIEmbeddings()

In [31]:
from pinecone import Pinecone

In [25]:
pinecone_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_key)

In [26]:
index = pc.Index("indchatpdf")

In [15]:
index_name = "indchatpdf"
for i, t in zip(range(len(text_chunks)), text_chunks):
   query_result = embedding.embed_query(t.page_content)
   index.upsert(
   vectors=[
        {
            "id": str(i),  # Convert i to a string
            "values": query_result, 
            "metadata": {"text":str(text_chunks[i].page_content)} # meta data as dic
        }
    ],
    namespace="real")
   index.describe_index_stats()

In [33]:
from langchain.vectorstores import Pinecone

text_field = 'text'
vectorstore = Pinecone(
    index, embedding, text_field
)

In [50]:
import openai
query = "What is a Transformer?"

res = openai.Embedding.create(
input=[query],
model="text-embedding-ada-002"
)

xq = res['data'][0]['embedding']

dres = index.query(vector=xq, top_k=2, namespace='real' , include_metadata=True)

In [63]:
limit = 3750

import time

def retrieve(query):
    res = openai.Embedding.create(
        input=[query],
        model = "text-embedding-ada-002"
    )

    # retrieve from Pinecone
    xq = res['data'][0]['embedding']

    # get relevant contexts
    contexts = []
    time_waited = 0
    while (len(contexts) < 3 and time_waited < 60 * 12):
        res = index.query(vector=xq, top_k=3,namespace='real', include_metadata=True)
        contexts = contexts + [
            x['metadata']['text'] for x in res['matches']
        ]
        print(f"Retrieved {len(contexts)} contexts, sleeping for 15 seconds...")
        time.sleep(15)
        time_waited += 15

    if time_waited >= 60 * 12:
        print("Timed out waiting for contexts to be retrieved.")
        contexts = ["No contexts retrieved. Try to answer the question yourself!"]


    # build our prompt with the retrieved contexts included
    prompt_start = (
        "Answer the question based on the context below.\n\n"+
        "Context:\n"
    )
    prompt_end = (
        f"\n\nQuestion: {query}\nAnswer:"
    )
    # append contexts until hitting limit
    for i in range(1, len(contexts)):
        if len("\n\n---\n\n".join(contexts[:i])) >= limit:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts[:i-1]) +
                prompt_end
            )
            break
        elif i == len(contexts)-1:
            prompt = (
                prompt_start +
                "\n\n---\n\n".join(contexts) +
                prompt_end
            )
    return prompt


def complete(prompt):
    # instructions
    sys_prompt = "You are a helpful assistant that always answers questions."
    # query text-davinci-003
    res = openai.ChatCompletion.create(
        model='gpt-3.5-turbo-0613',
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": prompt}
        ],
        temperature=0
    )
    return res['choices'][0]['message']['content'].strip()

In [64]:
query = (
    "What is encoder " +
    "in the transformer architecture"
)

query_with_contexts = retrieve(query)
query_with_contexts

Retrieved 3 contexts, sleeping for 15 seconds...


'Answer the question based on the context below.\n\nContext:\nFigure 1: The Transformer - model architecture.\nThe Transformer follows this overall architecture using stacked self-attention and point-wise, fully\nconnected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,\nrespectively.\n3.1 Encoder and Decoder Stacks\nEncoder: The encoder is composed of a stack of N= 6 identical layers. Each layer has two\nsub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, position-\n\n---\n\naligned RNNs or convolution. In the following sections, we will describe the Transformer, motivate\nself-attention and discuss its advantages over models such as [17, 18] and [9].\n3 Model Architecture\nMost competitive neural sequence transduction models have an encoder-decoder structure [ 5,2,35].\nHere, the encoder maps an input sequence of symbol representations (x1, ..., x n)to a sequence\nof continuous representations z= (z1, ..

In [65]:
complete(query_with_contexts)

'In the Transformer architecture, the encoder is a stack of N=6 identical layers. Each layer consists of two sub-layers: a multi-head self-attention mechanism and a simple, position-aligned RNNs or convolution.'

In [67]:
import sys
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = complete(retrieve(user_input))
  print(result)

Retrieved 3 contexts, sleeping for 15 seconds...
In the Transformer architecture, the encoder is a stack of N=6 identical layers. Each layer consists of two sub-layers: a multi-head self-attention mechanism and a simple, position-aligned RNNs or convolution.
Retrieved 3 contexts, sleeping for 15 seconds...
A decoder is a component in a neural sequence transduction model that generates an output sequence based on the continuous representations obtained from the encoder. It consists of a stack of identical layers, with each layer having two sub-layers and an additional sub-layer for multi-head attention over the output of the encoder stack. The decoder also employs residual connections and layer normalization for each sub-layer.
Exiting


SystemExit: 

C:\Users\rachi\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
